In [1]:
import os
import numpy as np
import requests
import concurrent.futures
import pandas as pd

In [2]:
def request_save(url, save_fp):
    img_data = requests.get(url, timeout=5).content
    with open(save_fp, 'wb') as handler:
        handler.write(img_data)

In [3]:
# parameters
class MyArgs():
    def __init__(self) -> None:
        self.partitions = 4
        self.part = 0
        self.data_dir = 'data/WebVid'
        self.csv_path ='data/WebVid/release/results_2M_train.csv'
        self.processes = 8

In [4]:
args = MyArgs()
# split dataset
### preproc
video_dir = os.path.join(args.data_dir, 'videos')
if not os.path.exists(os.path.join(video_dir, 'videos')):
    os.makedirs(os.path.join(video_dir, 'videos'))

# ASSUMES THE CSV FILE HAS BEEN SPLIT INTO N PARTS
partition_dir = args.csv_path.replace('.csv', f'_{args.partitions}')
# if not, then split in this job.
if not os.path.exists(partition_dir):
    os.makedirs(partition_dir)
    full_df = pd.read_csv(args.csv_path)
    df_split = np.array_split(full_df, args.partitions)
    for idx, subdf in enumerate(df_split):
        subdf.to_csv(os.path.join(partition_dir, f'{idx}.csv'), index=False)

relevant_fp = os.path.join(args.data_dir, 'relevant_videos_exists.txt')
if os.path.isfile(relevant_fp):
    exists = pd.read_csv(os.path.join(args.data_dir, 'relevant_videos_exists.txt'), names=['fn'])
else:
    exists = []

In [5]:
# df = pd.read_csv(os.path.join(partition_dir, f'{args.part}.csv'))
df = pd.read_csv(args.csv_path)
l = len(df)
# df[df.isnull().values==True]
df = df.dropna(axis=0,how='any')
l_a = len(df)
print(l_a, l - l_a)

2487525 367


In [11]:
df['rel_fn'] = df.apply(lambda x: os.path.join(x['page_dir'], str(x['videoid'])), axis=1)
df['rel_fn'] = df['rel_fn'] + '.mp4'
df = df[~df['rel_fn'].isin(exists)]
df.head()

,videoid,name,page_idx,page_dir,duration,contentUrl,rel_fn
0,31353427,"Merida, mexico - may 23, 2017: tourists are wa...",19972,016401_016450,PT00H00M15S,https://ak.picdn.net/shutterstock/videos/31353...,016401_016450/31353427
1,31221946,Fun clown - 3d animation,23313,023301_023350,PT00H00M19S,https://ak.picdn.net/shutterstock/videos/31221...,023301_023350/31221946
2,14781349,Happy family using laptop on bed at home,15856,015851_015900,PT00H00M13S,https://ak.picdn.net/shutterstock/videos/14781...,015851_015900/14781349
3,25054445,"11th march 2017. nakhon pathom, thailand. devo...",67495,067451_067500,PT00H00M06S,https://ak.picdn.net/shutterstock/videos/25054...,067451_067500/25054445
4,1011030806,Decorate with pineapple sweet cake roll.,7058,069551_069600,PT00H00M06S,https://ak.picdn.net/shutterstock/videos/10110...,069551_069600/1011030806


In [13]:
playlists_to_dl = np.sort(df['page_dir'].unique())
len(playlists_to_dl)

3590

In [14]:
for page_dir in playlists_to_dl:
    vid_dir_t = os.path.join(video_dir, page_dir)
    pdf = df[df['page_dir'] == page_dir]
    if len(pdf) > 0:
        if not os.path.exists(vid_dir_t):
            os.makedirs(vid_dir_t)

        urls_todo = []
        save_fps = []

        for idx, row in pdf.iterrows():
            video_fp = os.path.join(vid_dir_t, str(row['videoid']) + '.mp4')

            urls_todo.append(row['contentUrl'])
            save_fps.append(video_fp)

        print(f'Spawning {len(urls_todo)} jobs for page {page_dir}')
        with concurrent.futures.ThreadPoolExecutor(max_workers=args.processes) as executor:
            future_to_url = {executor.submit(request_save, url, fp) for url, fp in zip(urls_todo, save_fps)}

Spawning 4666 jobs for page 000001_000050
Spawning 4282 jobs for page 000051_000100
Spawning 3476 jobs for page 000101_000150
Spawning 2440 jobs for page 000151_000200
Spawning 2556 jobs for page 000201_000250
Spawning 2665 jobs for page 000251_000300
Spawning 3536 jobs for page 000301_000350
Spawning 3477 jobs for page 000351_000400
Spawning 2688 jobs for page 000401_000450
Spawning 2266 jobs for page 000451_000500
Spawning 2699 jobs for page 000501_000550
Spawning 3500 jobs for page 000551_000600
Spawning 2442 jobs for page 000601_000650
Spawning 3158 jobs for page 000651_000700
Spawning 2433 jobs for page 000701_000750
Spawning 2979 jobs for page 000751_000800
Spawning 2791 jobs for page 000801_000850
Spawning 2199 jobs for page 000851_000900
Spawning 3103 jobs for page 000901_000950
Spawning 2076 jobs for page 000951_001000
Spawning 2792 jobs for page 001001_001050
Spawning 3421 jobs for page 001051_001100
Spawning 2456 jobs for page 001101_001150
Spawning 3486 jobs for page 001151